### Testing model accuracy against textblob and Vadersentiment

#### First test against amazon review test data

In [1]:
from textblob import TextBlob
import pandas as pd

In [2]:
test_set = pd.read_csv("Data/amazon_review_polarity/test.csv")

In [3]:
test_set.columns = ['sentiment', 'title', 'review']

In [4]:
def recode(val):
    if val == 1:
        return 0
    else:
        return 1
    
def recode_s140(val):
    if val == 4:
        return 1
    elif val == 0:
        return 0

In [5]:
test_set['sentiment'] = test_set['sentiment'].apply(lambda x: recode(x))

In [6]:
sample_size = 4096*4
sample = test_set.sample(sample_size)

In [7]:
sample.head()

,sentiment,title,review
32566,0,no good for just 1 person,"I got this as a gift, and while it looked real..."
80457,0,Not soft/quality,"This is not a typical ""Sateen"" product. It isn..."
118976,1,Very impressed,I have to say I am very impressed with this bo...
267931,1,Stands up to beach side abuses.,After burning through a number of vacuums with...
87743,0,"Traditional, old school, disappointing",What a disappointment. This book reads as if i...


In [8]:
sample['text_blob_polarity'] = sample['review'].apply(lambda x: TextBlob(x).polarity)

In [9]:
def zero_one(x):
    return round((x + 1)/2)

In [10]:
sample['text_blob_polarity_round'] = sample['text_blob_polarity'].apply(lambda x: zero_one(x))

In [11]:
import numpy as np

test_score = 0.0
sentiments1 = np.array(sample['sentiment'])
sentiments_pred = np.array(sample['text_blob_polarity_round'])

In [12]:
def test_score(target, pred):
    num_correct = 0.0
    observations = len(target)
    for i in range(0, len(target)):
        if target[i] == pred[i]:
            num_correct += 1

    final_score = num_correct/observations
    return final_score

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [14]:
# function to print sentiments
# of the sentence.
# Create a SentimentIntensityAnalyzer object.
sid_obj = SentimentIntensityAnalyzer()
def sentiment_polarity(sentence):
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

In [15]:
sample['vader_polarity'] = sample['review'].apply(lambda x: sentiment_polarity(x))

In [16]:
sample['vader_polarity_round'] = sample['vader_polarity'].apply(lambda x: zero_one(x))

In [17]:
import pandas as pd
import numpy as np
import torch
%run DataPrep.ipynb
%run models/SentimentCNN_model.ipynb
%run TrainTestSentimentCNN.ipynb

CUDA is not available.  Training on CPU ...


In [18]:
import json

with open('Data/combined_data/combined_vocab_to_int.json', 'r') as vi:
    vocab_to_int = json.load(vi)

### Preprocessing amzon pol test data to feed model

In [19]:
# reviews_list = sample['review'].to_list()

In [20]:
# preprocessed_reviews_list = [preprocess(review) for review in reviews_list]

In [21]:
# # simple tokenization version - no lemmatization
# tokenized_text = ' '.join(preprocessed_reviews_list).split()

In [25]:
## Parameters
# try:
#     vocab_size = len(vocab_int) + 1 # for the 0 padding + our word tokens
# except:
#     vocab_size = vocab_size

# batch_size = 4096
vocab_size = 1559978
embedding_size = 300
seq_length = 70
output_size = 1

#### Bring in Model

In [26]:
# change the batch_size parameter to 1 when predicting.

model = SentimentCNN(vocab_size, output_size, embedding_size, 1, seq_length) # chan
print(model)

SentimentCNN(
  (embedding): Embedding(1559978, 300, padding_idx=0)
  (conv1): Conv1d(70, 64, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 16, kernel_size=(3,), stride=(1,))
  (conv4): Conv1d(16, 8, kernel_size=(3,), stride=(1,))
  (avgpool): AvgPool1d(kernel_size=(94,), stride=(94,), padding=(0,))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=8, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [27]:
# load the model with the trained parameters/weight that performed best in validation.
model.load_state_dict(torch.load('Sentiment_CNN_combined.pt'))

<All keys matched successfully>

#### Testing scores on amazon pol dataset

In [102]:
def predict_sentiment(model, new_texts, vocab_to_int, seq_length=40):
    """
    Function that takes in text, preproceses and passes it to the model for forward pass.
    Args: 
     - model to perform the inference
     - input text
     - word to integer mapping dict
     - sequence length the text is padded to
    :Returns a score of positive or negative."""
    
    model.eval()
    
    # preprocess, tokenize and lemmatize review
    new_texts = preprocess(new_texts)
    new_texts_ints = word_to_int2(new_texts, vocab_to_int, token_lem=True)
    
    # pad tokenized sequence
    features = np.zeros((seq_length), dtype=int)
    if features.shape[0] >= len(new_texts_ints):
        features[seq_length-len(new_texts_ints):] = np.array(new_texts_ints)[:seq_length]
    else:
        features[:] = np.array(new_texts_ints)[:seq_length]
    
    input_tensor = torch.from_numpy(features)
    
    # perform a forward pass from the model
    output = model(input_tensor)

    pred = output.detach().numpy()[0][0]
    if pred >= 0.55:
        return ("positive, {:.4f}".format(2*pred - 1))
    elif pred < 0.45: 
        return ("negative, {:.4f}".format(2*pred - 1))
    else:
        return ("neutral, 0.000")

In [48]:
def predict_class(model, new_texts, vocab_to_int, seq_length=40):
    """
    Function that takes in text, preproceses and passes it to the model for forward pass.
    Args: 
     - model to perform the inference
     - input text
     - word to integer mapping dict
     - sequence length the text is padded to
    :Returns a score of positive or negative."""
    
    model.eval()
    
    # preprocess, tokenize and lemmatize review
    new_texts = preprocess(new_texts)
    new_texts_ints = word_to_int(new_texts, vocab_to_int, token_lem=True)
    
    # pad tokenized sequence
    features = np.zeros((seq_length), dtype=int)
    if features.shape[0] >= len(new_texts_ints):
        features[seq_length-len(new_texts_ints):] = np.array(new_texts_ints)[:seq_length]
    else:
        features[:] = np.array(new_texts_ints)[:seq_length]
    
    input_tensor = torch.from_numpy(features)
    
    # perform a forward pass from the model
    output = model(input_tensor)

    pred = output.detach().numpy()[0][0]
    if pred >= 0.5:
        return 1
    elif pred < 0.5: 
        return 0

In [30]:
t = 'wow, that was so wonderful at first, but I really hated the beginning'

In [31]:
def word_to_int2(input_text: iter, vocab_to_int: dict, token_lem=False) -> iter:
    """
    A function to be used for encoding text to integers for prediction (assigning unknown words).
    Return: list of integers representing words in text
    """
    if token_lem == True:
        standardised_text = tokenize_lemmatize(input_text)
    else:
        standardised_text = input_text.split()
    # Convert words not in lookup to '<unk>'.
    
    word_ints = []
    for ii, word in enumerate(standardised_text):
        try:
            word_ints.append(vocab_to_int[word])
        except KeyError:
            word_ints.append(1)
        # assign the text integer values.   
    return word_ints

In [33]:
sample['model_pred'] = sample['review'].apply(lambda x: class_predict(model, x, vocab_to_int, seq_length=seq_length))

In [34]:
print("model acc score: {:.3f}".format(test_score(sentiments1, sample['model_pred'].to_list())))
print("Vader sentiment acc score: {:.3f}".format(test_score(sentiments1, sample['vader_polarity_round'].to_list())))
print("text blob acc score: {:.3f}".format(test_score(sentiments1, sentiments_pred)))

model acc score: 0.739
Vader sentiment acc score: 0.695
text blob acc score: 0.668


### Testing on IMDB movies review dataset

In [36]:
test_set2 = pd.read_csv("Data/combined_IMDB_reviews_data.csv").sample(sample_size)

In [37]:
test_set2['text_blob_polarity'] = test_set2['review'].apply(lambda x: TextBlob(x).polarity)
test_set2['text_blob_polarity_round'] = test_set2['text_blob_polarity'].apply(lambda x: zero_one(x))

In [38]:
sentiments2 = np.array(test_set2['sentiment'])
textblob_pred = np.array(test_set2['text_blob_polarity_round'])

In [39]:
test_set2['vader_polarity'] = test_set2['review'].apply(lambda x: sentiment_polarity(x))
test_set2['vader_polarity_round'] = test_set2['vader_polarity'].apply(lambda x: zero_one(x))

In [40]:
test_set2['model_pred'] = test_set2['review'].apply(lambda x: class_predict(model, x, vocab_to_int, seq_length=seq_length))

In [41]:
print("model acc score: {:.3f}".format(test_score(sentiments2, test_set2['model_pred'].to_list())))
print("Vader sentiment acc score: {:.3f}".format(test_score(sentiments2, test_set2['vader_polarity_round'].to_list())))
print("text blob final score: {:.3f}".format(test_score(sentiments2, textblob_pred)))

model acc score: 0.731
Vader sentiment acc score: 0.697
text blob final score: 0.690


### Test on Watson Chatbot Data

In [112]:
chatbot_data = pd.read_csv("Data/watson_output_combined.csv")

In [113]:
def chatbot_model_pred(x: str, pred_model):
    tokenized = x.split()
    models = ['cnn', 'vader', 'textblob']
    
    if len(tokenized) < 2 or '?' in x:
        pass
    else:
        if pred_model == models[0]:
            return predict_sentiment(model, x, vocab_to_int, seq_length=seq_length)
        elif pred_model == models[1]:
            return sid_obj.polarity_scores(x)
        elif pred_model == models[2]:
            return text_blob_pred(x)
        else:
            print("Assign the correct model")
            raise ValueError
        

In [114]:
def chatbot_hybrid_pred(x: str):
    tokenized = x.split()
    
    if len(tokenized) < 2:
        pass
    else:
        if '?' in x:
            return sid_obj.polarity_scores(x)
        else:
            return predict_sentiment(model, x, vocab_to_int, seq_length=seq_length)

In [115]:
chatbot_data = chatbot_data.drop(labels=['Unnamed: 0'], axis=1)

In [116]:
chatbot_data['hybrid_pred'] = chatbot_data['User Input'].apply(lambda x: chatbot_hybrid_pred(x))

In [117]:
chatbot_data.head()

,conversation_id,request_timestamp,response_timestamp,User Input,Output,Intent,Confidence,Exit Reason,Logging,Context,hybrid_pred
0,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:33:47+00:00,2021-07-07 18:33:49.552000+00:00,what are the store's opening hours?,Our hours are Monday to Friday 10am to 8pm and...,Customer_Care_Store_Hours,0.992556,completed,NaN,"{""system"": {""dialog_turn_counter"": 1, ""dialog_...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:02.029000+00:00,2021-07-07 18:34:02.106000+00:00,what are the store's opening hours?,I can tell you about our store locations and o...,Help,1.000000,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:27.870000+00:00,2021-07-07 18:34:28.004000+00:00,my delivery is late,Our hours are Monday to Friday 10am to 8pm and...,Customer_Care_Store_Hours,0.220531,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.4695"
3,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:38.932000+00:00,2021-07-07 18:34:39.076000+00:00,I want to cancel my order,I didn't understand can you try again,Cancel,0.545665,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.9401"
4,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:45.975000+00:00,2021-07-07 18:34:46.051000+00:00,Connect to agent,Would you like me to transfer you to a represe...,General_Connect_to_Agent,0.983706,NaN,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.2726"


In [105]:
chatbot_data['vader_pred'] = chatbot_data['User Input'].apply(lambda x: chatbot_model_pred(x, 'vader')) 

In [106]:
def text_blob_pred(x):
    """ """
    polarity_score = TextBlob(x).polarity
    if polarity_score > 0.1:
        return ("positive, {:.4f}".format(polarity_score))
    elif polarity_score < -0.1:
        return ("negative, {:.4f}".format(polarity_score))
    else:
        return ("neutral, {:.4f}".format(polarity_score))

In [107]:
chatbot_data['textblob_pred'] = chatbot_data['User Input'].apply(lambda x: chatbot_model_pred(x, 'textblob'))

In [108]:
chatbot_data.head()

,conversation_id,request_timestamp,response_timestamp,User Input,Output,Intent,Confidence,Exit Reason,Logging,Context,model_pred,vader_pred,textblob_pred
0,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:33:47+00:00,2021-07-07 18:33:49.552000+00:00,what are the store's opening hours?,Our hours are Monday to Friday 10am to 8pm and...,Customer_Care_Store_Hours,0.992556,completed,NaN,"{""system"": {""dialog_turn_counter"": 1, ""dialog_...",None,None,None
1,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:02.029000+00:00,2021-07-07 18:34:02.106000+00:00,what are the store's opening hours?,I can tell you about our store locations and o...,Help,1.000000,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...",None,None,None
2,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:27.870000+00:00,2021-07-07 18:34:28.004000+00:00,my delivery is late,Our hours are Monday to Friday 10am to 8pm and...,Customer_Care_Store_Hours,0.220531,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.4695","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","negative, -0.3000"
3,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:38.932000+00:00,2021-07-07 18:34:39.076000+00:00,I want to cancel my order,I didn't understand can you try again,Cancel,0.545665,completed,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.9401","{'neg': 0.274, 'neu': 0.548, 'pos': 0.178, 'co...","neutral, 0.0000"
4,04aa7771-37ba-4e8d-a993-336899cc1ede,2021-07-07 18:34:45.975000+00:00,2021-07-07 18:34:46.051000+00:00,Connect to agent,Would you like me to transfer you to a represe...,General_Connect_to_Agent,0.983706,NaN,NaN,"{""system"": {""session_id"": ""04aa7771-37ba-4e8d-...","negative, -0.2726","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","neutral, 0.0000"


In [118]:
chatbot_data.to_csv("Data/watson_data_pred_output_hybrid.csv")

### Testing on Yelp reviews dataset

In [57]:
yelp_data = pd.read_csv("Data/yelp_review_polarity_csv/test.csv")

In [58]:
yelp_data.head()

,2,"Contrary to other reviews, I have zero complaints about the service or the prices. I have been getting tire service here for the past 5 years now, and compared to my experience with places like Pep Boys, these guys are experienced and know what they're doing. \nAlso, this is one place that I do not feel like I am being taken advantage of, just because of my gender. Other auto mechanics have been notorious for capitalizing on my ignorance of cars, and have sucked my bank account dry. But here, my service and road coverage has all been well explained - and let up to me to decide. \nAnd they just renovated the waiting room. It looks a lot better than it did in previous years."
0,1,Last summer I had an appointment to get new ti...
1,2,"Friendly staff, same starbucks fair you get an..."
2,1,The food is good. Unfortunately the service is...
3,2,Even when we didn't have a car Filene's Baseme...
4,2,"Picture Billy Joel's \""Piano Man\"" DOUBLED mix..."


In [59]:
yelp_data.columns = ['sentiment', 'review']

In [60]:
yelp_data['sentiment'] = yelp_data['sentiment'].apply(lambda x: recode(x))

In [61]:
test_data3 = yelp_data.sample(sample_size)

In [62]:
test_data3['text_blob_polarity'] = test_data3['review'].apply(lambda x: TextBlob(x).polarity)
test_data3['text_blob_polarity_round'] = test_data3['text_blob_polarity'].apply(lambda x: zero_one(x))

In [63]:
sentiments3 = np.array(test_data3['sentiment'])
textblob_pred = np.array(test_data3['text_blob_polarity_round'])

In [64]:
test_data3['vader_polarity'] = test_data3['review'].apply(lambda x: sentiment_polarity(x))
test_data3['vader_polarity_round'] = test_data3['vader_polarity'].apply(lambda x: zero_one(x))

In [66]:
test_data3['model_pred'] = test_data3['review'].apply(lambda x: class_predict(model, x, vocab_to_int, seq_length=seq_length))

In [67]:
print("model acc score: {:.3f}".format(test_score(sentiments3, test_data3['model_pred'].to_list())))
print("Vader sentiment acc score: {:.3f}".format(test_score(sentiments3, test_data3['vader_polarity_round'].to_list())))
print("text blob final score: {:.3f}".format(test_score(sentiments3, textblob_pred)))

model acc score: 0.767
Vader sentiment acc score: 0.710
text blob final score: 0.687


#### Testing on Sentiment140 reviews dataset

In [69]:
s140 = pd.read_csv("Data/Sentiment140/test.csv")

In [70]:
s140.head()

,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right."
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [71]:
s140.columns = ['polarity', 'id', 'date', 'query', 'user', 'tweet']

In [72]:
s140.head()

,polarity,id,date,query,user,tweet
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [73]:
s140 = s140[s140['polarity'] != 2]

In [74]:
s140['polarity'] = s140['polarity'].apply(lambda x: recode_s140(x))

In [75]:
s140.head()

,polarity,id,date,query,user,tweet
0,1,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,1,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,1,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,1,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,1,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [76]:
from string import punctuation

def preprocess_tweet(tweet: str) -> str:
    return ''.join(c for c in tweet if c not in punctuation)

In [79]:
preprocess_tweet(t)

'kenburbary Youll love your Kindle2 Ive had mine for a few months and never looked back The new big one is huge No need for remorse '

In [80]:
s140['preprocessed_tweet'] = s140['tweet'].apply(lambda x: preprocess_tweet(x))

In [81]:
s140['text_blob_polarity'] = s140['preprocessed_tweet'].apply(lambda x: TextBlob(x).polarity)
s140['text_blob_polarity_round'] = s140['text_blob_polarity'].apply(lambda x: zero_one(x))

In [82]:
sentiments4 = np.array(s140['polarity'])
textblob_pred = np.array(s140['text_blob_polarity_round'])

In [83]:
s140['vader_polarity'] = s140['preprocessed_tweet'].apply(lambda x: sentiment_polarity(x))
s140['vader_polarity_round'] = s140['vader_polarity'].apply(lambda x: zero_one(x))

In [86]:
s140['model_pred'] = s140['preprocessed_tweet'].apply(lambda x: class_predict(model, x, vocab_to_int, seq_length=seq_length))

In [87]:
print("model acc score: {:.3f}".format(test_score(sentiments4, s140['model_pred'].to_list())))
print("Vader sentiment acc score: {:.3f}".format(test_score(sentiments4, s140['vader_polarity_round'].to_list())))
print("text blob final score: {:.3f}".format(test_score(sentiments4, textblob_pred)))

model acc score: 0.763
Vader sentiment acc score: 0.791
text blob final score: 0.715


In [88]:
s140[['polarity', 'preprocessed_tweet','text_blob_polarity', 'text_blob_polarity_round', 'vader_polarity', 'vader_polarity_round', 'model_pred']].to_csv("s140_pred_output.csv")